In [ ]:
#This is used to crop video and sensor data from a session file.
#This makes individual trial sensor/pixel data files, sorted into forward and backward runs. 

#After you have the cropped .mat files from this script, use batch_cropVidReader script to extract video.

#Delete original session .mat file, compress videos (ffmpeg), and remove pixel data (remove_pixel script) from 
#trial files to complete compression.  

In [ ]:
#dependencies
import numpy as np
import os,sys
import h5py
import hdf5storage
import matplotlib.pyplot as plt 
import matplotlib as mpl
import pylab
import pandas as pd

In [ ]:
#variables
name = '.mat'
fps = 30

In [ ]:
#function to identify 1 location in array 
def one_runs(a):
    # Create an array that is 1 where a is 1, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 1).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

#function to identify 0 location in array 
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges


In [ ]:
f = h5py.File(name, 'r')

allSensor = []
allPixel = []

#shape is approximating number frames 
struArray = f['data/frame']
shape = struArray['sns'].shape
shape = shape[0] - 1 

while shape > 0: 
    value = f[struArray['sns'][shape,0]].value 
    pixel = f[struArray['img'][shape,0]].value
    allSensor.append(value)
    allPixel.append(pixel)
    shape = shape - 1 
            
#remove extraneous array bracket 
frames = np.array([e[0] for e in allSensor])

#find location of 1 runs
OnesLoc = []
for array in frames.T: 
    OnesLoc.append(one_runs(array))
        
#remove never active sensors (empty arrays) 
#find frames at which sensor was first and last active
touches = []
sensorID = []
frame1 = [] 
frame2 = []

for i,v in enumerate(OnesLoc):
    if len(v) != 0:
        for thing in v: 
            touches.append(thing)
            sensorID.append(i)
            frame1.append(thing[0])
            frame2.append(thing[1])
        else: 
            pass
        
#find duration in seconds for which each sensor was active 
duration = ((np.asarray(frame1) - np.asarray(frame2))* -1)/fps

#make dataframe and sort by the first frame at which the sensor active 
d = {'sensorID': sensorID, 'frame': touches, 'frame1' : frame1, 'frame2' : frame2, 'duration': duration,}
df = pd.DataFrame(data=d)
df = df.sort_values(by=['frame1'])

#find difference in sensors activated, padding with zero at the beginning 
sensorDiff = np.diff(df['sensorID'].values)
sensorDiff = np.insert(sensorDiff, 0,0)

#smooth sensorDiff: important to reduce noise, and keep values above or below zero
#add sensor difference to dataframe
smoothingKernel = 5
kernel = np.ones(smoothingKernel)
conv1 = np.convolve(sensorDiff, kernel)
conv11 = [x/smoothingKernel for x in conv1]
df['sensorDiff'] = np.asarray(conv11[2:(len(conv11)-2)])


#for each value in sensorDiff,   if the difference is positive, assume mouse is going forward (1)
#                                if the difference is negative, assume mouse is going backward (0)
#                                add direction column to dataframe

direction = []
for x in df['sensorDiff'].values: 
    if x < 0: #backward
        direction.append(0)
    elif x > 0: #forward
        direction.append(1)
    else: 
        direction.append(None)
        
df['direction'] = direction
df = df.reset_index()


In [ ]:
#find the row min and max for each forward run (run of 1's) and backward run(run of 0's)
forwardruns = one_runs(df['direction'].values)
backwardruns = zero_runs(df['direction'].values)

In [ ]:
#for each forward run, use the first and last frame of that run to crop out the relevant video and sensor data
trial = 0 
for array in forwardruns: 
    firstframe = df['frame1'][array[0]]
    lastframe = df['frame2'][array[1]-1]
    if ((array[1] - array[0]) > 15):  
            cropPixel = allPixel[firstframe:lastframe]
            cropSensor = allSensor[firstframe:lastframe]
            
            trial = trial + 1
            matfiledata = {} # make a dictionary to store the MAT data in

            # *** u prefix for variable name = unicode format, no issues thru Python 3.5; advise keeping u prefix indicator format based on feedback despite docs ***
            matfiledata[u'img'] = cropPixel
            matfiledata[u'sns'] = cropSensor
            hdf5storage.savemat(str(trial) + '_forward' + '.mat', matfiledata, format='7.3', oned_as='column')
    else: 
        pass
    
#for each backward run, use the first and last frame of that run to crop out the relevant video and sensor data    
trial = 0 
for array in backwardruns: 
    firstframe = df['frame1'][array[0]]
    lastframe = df['frame2'][array[1]-1]
    if array[1] - array[0] > 15:  
            cropPixel = allPixel[firstframe:lastframe]
            cropSensor = allSensor[firstframe:lastframe]
            
            trial = trial + 1
            
            matfiledata = {} # make a dictionary to store the MAT data in

            # *** u prefix for variable name = unicode format, no issues thru Python 3.5; advise keeping u prefix indicator format based on feedback despite docs ***
            matfiledata[u'img'] = cropPixel
            matfiledata[u'sns'] = cropSensor
            hdf5storage.savemat(str(trial) + '_backward' + '.mat', matfiledata, format='7.3', oned_as='column')
            
            
            

In [ ]:
#files are saved in the folder in which you have this script

os.system('say "program complete"')